# Importing Gaia data and formatting for Uniview

*As of March 12, 2020 the astroquery.gaia TAP does NOT work*

So I downloaded the data with RV measurements from here: http://cdn.gea.esac.esa.int/Gaia/gdr2/gaia_source_with_rv/csv/


In [10]:
import astropy.units as u
from astropy.coordinates import ICRS, Distance
from astropy.coordinates.sky_coordinate import SkyCoord

import numpy as np
import pandas as pd

In [7]:
filelist = ['GaiaSource_2851858288640_1584379458008952960.csv',
            'GaiaSource_1584380076484244352_2200921635402776448.csv',
            'GaiaSource_2200921875920933120_3650804325670415744.csv',
            'GaiaSource_3650805523966057472_4475721411269270528.csv',
            'GaiaSource_4475722064104327936_5502601461277677696.csv',
            'GaiaSource_6714230465835878784_6917528443525529728.csv',
            'GaiaSource_5502601873595430784_5933051501826387072.csv']
ra = []
dec = []
parallax = []
pmra = []
pmdec = []
radial_velocity = []
teff = []
gmag = []
#also have column for flag for star types
for i, file in enumerate(filelist):
    print(file)
    d = pd.read_csv(file)
    ra.extend(d['ra'].values)
    dec.extend(d['dec'].values)
    parallax.extend(d['parallax'].values)
    pmra.extend(d['pmra'].values)
    pmdec.extend(d['pmdec'].values)
    radial_velocity.extend(d['radial_velocity'].values)
    teff.extend(d['teff_val'].values)
    gmag.extend(d['phot_g_mean_mag'].values)

GaiaSource_2851858288640_1584379458008952960.csv
GaiaSource_1584380076484244352_2200921635402776448.csv
GaiaSource_2200921875920933120_3650804325670415744.csv
GaiaSource_3650805523966057472_4475721411269270528.csv
GaiaSource_4475722064104327936_5502601461277677696.csv
GaiaSource_6714230465835878784_6917528443525529728.csv
GaiaSource_5502601873595430784_5933051501826387072.csv


In [8]:
#convert to numpy arrays
ra = np.array(ra)
dec = np.array(dec)
parallax = np.array(parallax)
pmra = np.array(pmra)
pmdec = np.array(pmdec)
radial_velocity = np.array(radial_velocity)
teff = np.array(teff)
gmag = np.array(gmag)

### Convert these ra, dec, parallax coordinates to 3D cartesian

In [17]:
#what about the negative parallax values?
dist = Distance(parallax=np.abs(parallax)*u.mas).to(u.parsec)


#NOTE: without RVs, the velocities are not correct
astroC = ICRS(ra = ra*u.deg, 
              dec = dec*u.deg, 
              distance = dist, 
              pm_ra_cosdec = pmra*u.mas/u.yr, 
              pm_dec = pmdec*u.mas/u.yr, 
              radial_velocity = radial_velocity* u.km/u.s )

print(astroC.cartesian)
print(astroC.velocity)

[( 269.38407612,   270.6295352 ,  9.18710930e-01),
 ( 405.43557556,   406.25961987,  1.27625851e+00),
 ( 124.13530814,   124.28418234,  7.21671565e-01), ...,
 (-528.17667664, -1053.87212127, -1.57007818e+03),
 (-848.39362534, -1692.04421689, -2.52007171e+03),
 (-676.93760028, -1353.1806852 , -2.01261360e+03)] pc
[( 17.68034972, 30.03302423,  34.09434331),
 ( -9.03826025, 15.59072114, -35.20460752),
 (-22.19402271, 31.25006863,  -5.38147725), ...,
 ( 14.08570307, 82.51525053,  28.47311102),
 (-15.5730583 , 25.36941381, -27.84399477),
 (  2.69895394, 41.68614988,  34.34951455)] km / s


### What should I use as a flag?

In [19]:
#will want to fill this in
flag = np.zeros_like(ra)

### Now put these in the "raw" format

In [ ]:
p = astroC.cartesian
v = astroC.velocity

x = p.x.to(u.pc).value
y = p.y.to(u.pc).value
z = p.z.to(u.pc).value

vx = v.d_x.to(u.km/u.s).value
vy = v.d_y.to(u.km/u.s).value
vz = v.d_z.to(u.km/u.s).value



dct = {'x':x,'y':y,'z':z,'vx':vx,'vy':vy,'vz':vz,'teff':teff,'gmag':gmag,'flag':flag}
df = pd.DataFrame(df) 

In [20]:
df.to_csv('GaiaRVsample.raw', index=False, sep=' ', header=False)